In [5]:
import pandas as pd
import os, sys, json
from tqdm import tqdm
root_dir = os.path.dirname(os.getcwd())
sys.path.append(root_dir)
import numpy as np

# Datasets & Baselines

In [6]:

metadataset_df = pd.read_csv(f"{root_dir}/TabZilla/tutorials/metadataset_new.csv")

def read_json(file):
    with open(file, "r") as f:
        return json.load(f)
    
eval = {}

model_dict = {
    'tabpfn': 'TabPFNModel',
    'tabflex': 'TabFlexModel',
    'tabfast': 'TabFastModel',
}

baselines = [
    'LinearModel', 
    'KNN', 
    'SVM', 
    'DecisionTree', 
    'RandomForest', 
    'XGBoost', 
    'CatBoost', 
    'LightGBM', 
    'MLP', 
    'TabNet', 
    'VIME', 
    'TabTransformer', 
    'NODE', 
    'DeepGBM', 
    'STG', 
    'NAM', 
    'DeepFM', 
    'SAINT', 
    'DANet', 
    'rtdl_MLP', 
    'rtdl_ResNet', 
    'rtdl_FTTransformer',
    'Ours-2',
    'TuneTables',
    'HyperFast',
]

In [7]:
def get_datasets(table_idx):
    if table_idx == 1:
        from analysis.table1 import DATASETS
    elif table_idx == 2:
        from analysis.table2 import DATASETS
    elif table_idx == 4:
        from analysis.table4 import DATASETS
        
    return DATASETS

In [8]:
def get_dataset_stat(dataset):
    result_dir = f"{root_dir}/results/TabFlexModel/{dataset}"
    items = os.listdir(result_dir)
    for item in items:
        if item == 'default_trial0_results.json':
            result = read_json(f"{result_dir}/{item}")
            break
        elif os.path.isdir(f"{result_dir}/{item}"):
            result = read_json(f"{result_dir}/{item}/default_trial0_results.json")
            break
    return {
        'num_classes': result['dataset']['num_classes'],
        'num_features': result['dataset']['num_features'],
        'num_instances': result['dataset']['num_instances'],
    }

In [9]:
def get_results(dataset, model, metric = 'Accuracy'):
    result_dir = f"{root_dir}/results/{model_dict[model]}/{dataset}"
    items = os.listdir(result_dir)
    
    best_eval = None
    for item in items: 
        try:
            if item == 'default_trial0_results.json':
                eval = read_json(f"{result_dir}/{item}")
                if best_eval is None or eval['scorers']['test'][metric] > best_eval['scorers']['test'][metric]:
                    best_eval = eval
            elif os.path.isdir(f"{result_dir}/{item}"):
                eval = read_json(f"{result_dir}/{item}/default_trial0_results.json")
                if best_eval is None or eval['scorers']['test'][metric] > best_eval['scorers']['test'][metric]:
                    best_eval = eval
        except KeyboardInterrupt:
            raise KeyboardInterrupt
        except Exception as e:
            # print the error message
            print(e)
            print(f"Error reading {result_dir}/{item}")
                  
    return best_eval

In [10]:
def get_tabmodels(method, eval, raw_results, metric = 'Accuracy'):
    eval[method] = {}
    for dataset in raw_results[method]:
        eval[method][dataset] = {}
        metrics = raw_results[method][dataset]['scorers']['test'][metric]
        metrics = pd.Series(metrics)
        eval[method][dataset][f'median_{metric}'] = metrics.median()
        eval[method][dataset][f'mean_{metric}'] = metrics.mean()
        eval[method][dataset]['train_time'] = pd.Series(raw_results[method][dataset]['timers']['train'][1:]).mean()
        eval[method][dataset]['test_time'] = pd.Series(raw_results[method][dataset]['timers']['test']).mean()
        eval[method][dataset]['total_time'] = eval[method][dataset]['train_time'] + eval[method][dataset]['test_time']
    

In [11]:
def get_baselines(method, eval, DATASETS, metric = 'Accuracy'):
    eval[method] = {}
    for dataset in DATASETS:
        eval[method][dataset] = {}
        result = metadataset_df.loc[
            (metadataset_df["alg_name"] == method) & 
            (metadataset_df["hparam_source"] == "default") &
            (metadataset_df["dataset_name"] == dataset),
            [
                "dataset_fold_id", 
                "alg_name", 
                "hparam_source", 
                f"{metric}__test", 
                "training_time",
                'eval-time__test',]
        ]
        if result.empty:
            eval[method][dataset][f'median_{metric}'] = 0
            eval[method][dataset][f'mean_{metric}'] = 0
            eval[method][dataset]['train_time'] = 0
            eval[method][dataset]['test_time'] = 0
            eval[method][dataset]['total_time'] = 0
        else:
            eval[method][dataset][f'median_{metric}'] = result[f"{metric}__test"].median()
            eval[method][dataset][f'mean_{metric}'] = result[f"{metric}__test"].mean()
            eval[method][dataset]['train_time'] = result['training_time'].mean()
            eval[method][dataset]['test_time'] = result['eval-time__test'].mean()
            eval[method][dataset]['total_time'] = eval[method][dataset]['train_time'] + eval[method][dataset]['test_time']

# Get the Results of TabModels

In [12]:
table_idx = 2
metric = 'AUC'

DATASETS = get_datasets(table_idx)
raw_results = {}

tqdm_bar = tqdm(model_dict)
for model in tqdm_bar:
    tqdm_bar.set_description(f"Reading results of {model}...")
    raw_results[model] = {}
    for dataset in DATASETS:
        raw_results[model][dataset] = get_results(dataset, model, metric = metric)            
        
tqdm_bar = tqdm(model_dict)
for model in tqdm_bar:
    tqdm_bar.set_description(f"Computing results of {model}...")
    get_tabmodels(model, eval, raw_results, metric)

Computing results of tabfast...: 100%|██████████| 3/3 [00:00<00:00, 59.35it/s]


In [13]:
table_dict = {}

for dataset in DATASETS:
    table_dict[dataset] = get_dataset_stat(dataset)
    for method in model_dict:
        table_dict[dataset][f'{method}_{metric}'] = round(eval[method][dataset][f'mean_{metric}'], 4)
        table_dict[dataset][f'{method}_time'] = round(eval[method][dataset]['total_time'], 4)
        
table = pd.DataFrame(table_dict).T

table

,num_classes,num_features,num_instances,tabpfn_AUC,tabpfn_time,tabflex_AUC,tabflex_time,tabfast_AUC,tabfast_time
openml__Australian__146818,1.0,14.0,690.0,0.9318,0.1900,0.9266,0.0971,0.9244,0.0218
openml__LED-display-domain-7digit__125921,10.0,7.0,500.0,0.9356,0.2360,0.9288,0.0423,0.9358,0.0402
openml__MiceProtein__146800,8.0,77.0,1080.0,1.0000,0.4658,0.9984,0.4020,0.9991,0.3129
openml__acute-inflammations__10089,1.0,6.0,120.0,1.0000,0.0432,1.0000,0.0376,1.0000,0.0356
openml__analcatdata_authorship__3549,4.0,70.0,841.0,1.0000,0.3781,1.0000,0.3599,1.0000,0.2802
openml__analcatdata_boxing1__3540,1.0,3.0,120.0,0.5793,0.0233,0.5544,0.0269,0.5492,0.0289
openml__analcatdata_chlamydia__3739,1.0,3.0,100.0,0.9688,0.0222,0.9750,0.0271,0.9938,0.0265
openml__analcatdata_dmft__3560,6.0,4.0,797.0,0.5875,0.1331,0.5871,0.0491,0.5815,0.0199
openml__anneal__2867,5.0,38.0,898.0,0.9955,0.2677,0.9901,0.1788,0.9895,0.1313
openml__autos__9,6.0,25.0,205.0,0.9534,0.1087,0.9165,0.0939,0.9388,0.0934


In [14]:
data_table = table[['num_features', 'num_instances', 'num_classes']].T
# for each column, only take the string between two "__"
data_table.columns = [col.split('__')[1] for col in data_table.columns]
data_table.T.sort_values(by='num_instances', ascending=False)

,num_features,num_instances,num_classes
yeast,8.0,1269.0,4.0
hill-valley,100.0,1212.0,1.0
socmob,5.0,1156.0,1.0
pc1,21.0,1109.0,1.0
MiceProtein,77.0,1080.0,8.0
qsar-biodeg,41.0,1055.0,1.0
tic-tac-toe,9.0,958.0,1.0
anneal,38.0,898.0,5.0
vehicle,18.0,846.0,4.0
analcatdata_authorship,70.0,841.0,4.0


In [17]:
# Convert float columns to integers
data_table_int = data_table.astype(int)

# Print the LaTeX table with integer values
print(data_table_int.T.to_latex(index=True, escape=True))

\begin{tabular}{lrrr}
\toprule
 & num\_features & num\_instances & num\_classes \\
\midrule
Australian & 14 & 690 & 1 \\
LED-display-domain-7digit & 7 & 500 & 10 \\
MiceProtein & 77 & 1080 & 8 \\
acute-inflammations & 6 & 120 & 1 \\
analcatdata\_authorship & 70 & 841 & 4 \\
analcatdata\_boxing1 & 3 & 120 & 1 \\
analcatdata\_chlamydia & 3 & 100 & 1 \\
analcatdata\_dmft & 4 & 797 & 6 \\
anneal & 38 & 898 & 5 \\
autos & 25 & 205 & 6 \\
balance-scale & 4 & 625 & 3 \\
blood-transfusion-service-center & 4 & 748 & 1 \\
blood-transfusion-service-center & 4 & 748 & 1 \\
breast-cancer & 9 & 286 & 1 \\
breast-w & 9 & 699 & 1 \\
colic & 26 & 368 & 1 \\
colic & 22 & 368 & 1 \\
credit-approval & 15 & 690 & 1 \\
cylinder-bands & 37 & 540 & 1 \\
dermatology & 34 & 366 & 6 \\
diabetes & 8 & 768 & 1 \\
dresses-sales & 12 & 500 & 1 \\
ecoli & 7 & 336 & 8 \\
eucalyptus & 19 & 736 & 5 \\
fertility & 9 & 100 & 1 \\
fri\_c0\_100\_5 & 5 & 100 & 1 \\
fri\_c3\_100\_5 & 5 & 100 & 1 \\
glass & 9 & 214 & 6 \\
haye

In [15]:
# Split the table into three parts
data_table_reset = data_table.T.reset_index()
data_table_reset = data_table_reset.sort_values(by='index', key=lambda x: x.str.len())
part1 = data_table_reset.iloc[:33].reset_index(drop=True)
part2 = data_table_reset.iloc[33:66].reset_index(drop=True)
part3 = data_table_reset.iloc[66:].reset_index(drop=True)

# Rename columns to avoid duplicates
part2.columns = [f'{col}_2' for col in part2.columns]
part3.columns = [f'{col}_3' for col in part3.columns]

# Concatenate the parts horizontally
combined_table = pd.concat([part1, part2, part3], axis=1)

# Convert digit columns to integers
for col in combined_table.columns:
    if combined_table[col].dtype == 'float64':
        combined_table[col] = combined_table[col].astype('Int64')

# Display the combined table
print(combined_table.to_latex(index=False, escape=True))

\begin{tabular}{lrrrlrrrlrrr}
\toprule
index & num\_features & num\_instances & num\_classes & index\_2 & num\_features\_2 & num\_instances\_2 & num\_classes\_2 & index\_3 & num\_features\_3 & num\_instances\_3 & num\_classes\_3 \\
\midrule
kc2 & 21 & 522 & 1 & dermatology & 34 & 366 & 6 & NaN & NaN & NaN & NaN \\
tae & 5 & 151 & 3 & tic-tac-toe & 9 & 958 & 1 & NaN & NaN & NaN & NaN \\
pc1 & 21 & 1109 & 1 & qsar-biodeg & 41 & 1055 & 1 & NaN & NaN & NaN & NaN \\
ilpd & 10 & 583 & 1 & lung-cancer & 56 & 32 & 3 & NaN & NaN & NaN & NaN \\
iris & 4 & 150 & 3 & fri\_c3\_100\_5 & 5 & 100 & 1 & NaN & NaN & NaN & NaN \\
wdbc & 30 & 569 & 1 & fri\_c0\_100\_5 & 5 & 100 & 1 & NaN & NaN & NaN & NaN \\
ecoli & 7 & 336 & 8 & balance-scale & 4 & 625 & 3 & NaN & NaN & NaN & NaN \\
irish & 5 & 500 & 1 & breast-cancer & 9 & 286 & 1 & NaN & NaN & NaN & NaN \\
colic & 22 & 368 & 1 & dresses-sales & 12 & 500 & 1 & NaN & NaN & NaN & NaN \\
colic & 26 & 368 & 1 & cylinder-bands & 37 & 540 & 1 & NaN & NaN & Na

In [11]:
table[(table[f'tabpfn_{metric}'] >= table[f'tabflex_{metric}']) & (table[f'tabpfn_{metric}'] >= table[f'tabfast_{metric}'])]

,num_classes,num_features,num_instances,tabpfn_AUC,tabpfn_time,tabflex_AUC,tabflex_time,tabfast_AUC,tabfast_time
openml__Australian__146818,1.0,14.0,690.0,0.9318,0.1900,0.9266,0.0971,0.9244,0.0218
openml__GesturePhaseSegmentationProcessed__14969,5.0,32.0,9873.0,0.8392,1.1072,0.7881,0.7250,0.8126,0.7722
openml__MiniBooNE__168335,1.0,50.0,130064.0,0.9769,3.1937,0.9708,7.8040,0.9740,7.2221
openml__artificial-characters__14964,10.0,7.0,10218.0,0.9601,0.9679,0.9211,0.3429,0.9500,0.2842
openml__balance-scale__11,3.0,4.0,625.0,0.9993,0.0999,0.9968,0.0410,0.9954,0.0190
openml__colic__25,1.0,26.0,368.0,0.8784,0.1427,0.8781,0.1262,0.8751,0.1030
openml__credit-approval__29,1.0,15.0,690.0,0.9353,0.1621,0.9298,0.0982,0.9243,0.0220
openml__credit-g__31,1.0,20.0,1000.0,0.7651,0.2235,0.7424,0.1325,0.7616,0.2307
openml__electricity__219,1.0,8.0,45312.0,0.8892,1.6534,0.8797,1.4924,0.8872,2.2287
openml__jungle_chess_2pcs_raw_endgame_complete__167119,3.0,6.0,44819.0,0.9466,1.6508,0.9200,1.3295,0.9269,1.1877


In [12]:
table[(table[f'tabflex_{metric}'] >= table[f'tabpfn_{metric}']) & (table[f'tabflex_{metric}'] >= table[f'tabfast_{metric}'])]

,num_classes,num_features,num_instances,tabpfn_AUC,tabpfn_time,tabflex_AUC,tabflex_time,tabfast_AUC,tabfast_time
openml__SpeedDating__146607,1.0,120.0,8378.0,0.5469,1.5849,0.8500,1.9131,0.8497,0.1289
openml__cnae-9__9981,9.0,856.0,1080.0,0.4794,0.5066,0.9558,3.7996,0.5028,0.9265
openml__guillermo__168337,1.0,4296.0,20000.0,0.4999,1.7556,0.5008,29.0964,0.4991,3.1121
openml__heart-h__50,1.0,13.0,294.0,0.8819,0.0837,0.8885,0.0663,0.8855,0.0194
openml__jasmine__168911,1.0,144.0,2984.0,0.7040,1.2732,0.8552,0.9882,0.8523,0.0737
openml__nomao__9977,1.0,118.0,34465.0,0.7623,1.8179,0.9884,5.3437,0.7568,2.7171
openml__one-hundred-plants-texture__9956,100.0,64.0,1599.0,0.5452,0.5155,0.5613,0.4049,0.5453,0.3066


In [13]:
table[(table[f'tabfast_{metric}'] >= table[f'tabpfn_{metric}']) & (table[f'tabfast_{metric}'] >= table[f'tabflex_{metric}'])]

,num_classes,num_features,num_instances,tabpfn_AUC,tabpfn_time,tabflex_AUC,tabflex_time,tabfast_AUC,tabfast_time
openml__Bioresponse__9910,1.0,1776.0,3751.0,0.4980,1.2862,0.4905,6.8685,0.5450,0.7605
openml__ada_agnostic__3896,1.0,48.0,4562.0,0.8863,1.0700,0.8921,0.4433,0.8958,0.3492
openml__airlines__189354,1.0,7.0,539383.0,0.6279,9.7280,0.6214,3.7808,0.6393,4.1948
openml__albert__189356,1.0,78.0,425240.0,0.6860,9.3899,0.6914,13.1747,0.7011,13.4606
openml__audiology__7,24.0,69.0,226.0,0.8182,0.2328,0.8143,0.2504,0.8222,0.8767
openml__elevators__3711,1.0,18.0,16599.0,0.9470,1.7780,0.9485,0.7805,0.9492,0.5989
openml__higgs__146606,1.0,28.0,98050.0,0.7241,2.8184,0.7014,6.2100,0.7560,4.9200
openml__poker-hand__9890,10.0,10.0,1025009.0,0.7176,15.3640,0.5559,4.5662,0.8372,4.8793
openml__profb__3561,1.0,9.0,672.0,0.6617,0.2318,0.6591,0.0675,0.6644,0.0527


In [14]:
table_fast = table[(table['num_instances'] >= 50000) & (table['num_features'] <= 100)].drop(columns = ['tabflex_time', f'tabflex_{metric}'])
table_fast

,num_classes,num_features,num_instances,tabpfn_AUC,tabpfn_time,tabfast_AUC,tabfast_time
openml__MiniBooNE__168335,1.0,50.0,130064.0,0.9769,3.1937,0.9740,7.2221
openml__airlines__189354,1.0,7.0,539383.0,0.6279,9.7280,0.6393,4.1948
openml__albert__189356,1.0,78.0,425240.0,0.6860,9.3899,0.7011,13.4606
openml__higgs__146606,1.0,28.0,98050.0,0.7241,2.8184,0.7560,4.9200
openml__poker-hand__9890,10.0,10.0,1025009.0,0.7176,15.3640,0.8372,4.8793


In [29]:
table_flex = table[(table['num_features'] > 100)].drop(columns = ['tabfast_time', f'tabfast_{metric}'])
table_flex

,num_classes,num_features,num_instances,tabpfn_AUC,tabpfn_time,tabflex_AUC,tabflex_time
openml__Bioresponse__9910,1.0,1776.0,3751.0,0.4980,1.2862,0.4905,6.8685
openml__SpeedDating__146607,1.0,120.0,8378.0,0.5469,1.5849,0.8500,1.9131
openml__cnae-9__9981,9.0,856.0,1080.0,0.4794,0.5066,0.9558,3.7996
openml__guillermo__168337,1.0,4296.0,20000.0,0.4999,1.7556,0.5008,29.0964
openml__jasmine__168911,1.0,144.0,2984.0,0.7040,1.2732,0.8552,0.9882
openml__nomao__9977,1.0,118.0,34465.0,0.7623,1.8179,0.9884,5.3437


# Compare with Other Baselines

In [16]:
tqdm_bar = tqdm(baselines)
for baseline in tqdm_bar:
    tqdm_bar.set_description(f"Computing results of {baseline}...") 
    try:
        get_baselines(baseline, eval, DATASETS, metric)
    except:
        print(f"Error in {baseline}")

methods = baselines + list(model_dict.keys())
final_results = {}

for method in methods:
    result = pd.DataFrame(eval[method]).T
    final_results[method] = result[f'mean_{metric}'].mean()
    
sorted_methods = sorted(methods, key=lambda x: final_results[x], reverse=True)
for method in sorted_methods:
    print(f"| {method} | {final_results[method]} |")

Computing results of HyperFast...: 100%|██████████| 25/25 [02:44<00:00,  6.60s/it]         

| XGBoost | 0.89626674151231 |
| CatBoost | 0.8710780319364649 |
| rtdl_ResNet | 0.865423274822542 |
| RandomForest | 0.8517725838051355 |
| rtdl_MLP | 0.8389464580267786 |
| tabfast | 0.8368820216658714 |
| tabflex | 0.8338746299932863 |
| tabpfn | 0.8225416153107362 |
| Ours-2 | 0.817822077242185 |
| MLP | 0.8144863457854493 |
| LightGBM | 0.8064431201510455 |
| DecisionTree | 0.7904649113211598 |
| LinearModel | 0.7844321715770098 |
| KNN | 0.7474054796492654 |
| NODE | 0.7247265454837093 |
| TabNet | 0.7219269236127556 |
| VIME | 0.688526351681953 |
| rtdl_FTTransformer | 0.6791009939981121 |
| STG | 0.6748985179731256 |
| DANet | 0.660580705297897 |
| SVM | 0.6507085992457963 |
| TabTransformer | 0.5891746586557063 |
| SAINT | 0.5367639924497778 |
| DeepFM | 0.5026993451840268 |
| NAM | 0.32484285783465794 |
| DeepGBM | 0.0 |
| TuneTables | 0.0 |
| HyperFast | 0.0 |


In [17]:
for dataset in DATASETS:
    table_dict[dataset] = get_dataset_stat(dataset)
    for method in baselines + list(model_dict.keys()):
        table_dict[dataset][f'{method}_{metric}'] = round(eval[method][dataset][f'mean_{metric}'], 4)
        table_dict[dataset][f'{method}_time'] = round(eval[method][dataset]['total_time'], 4)

In [18]:
full_table = pd.DataFrame(table_dict).T

In [19]:
full_table[full_table['num_features'] >= 1000]

,num_classes,num_features,num_instances,LinearModel_AUC,LinearModel_time,KNN_AUC,KNN_time,SVM_AUC,SVM_time,DecisionTree_AUC,...,TuneTables_AUC,TuneTables_time,HyperFast_AUC,HyperFast_time,tabpfn_AUC,tabpfn_time,tabflex_AUC,tabflex_time,tabfast_AUC,tabfast_time
openml__Bioresponse__9910,1.0,1776.0,3751.0,0.8093,0.2508,0.8059,12.7006,0.8521,208.5077,0.7797,...,0.0,0.0,0.0,0.0,0.4980,1.2862,0.4905,6.8685,0.5450,0.7605
openml__guillermo__168337,1.0,4296.0,20000.0,0.7069,3.6900,0.0000,0.0000,0.0000,0.0000,0.7455,...,0.0,0.0,0.0,0.0,0.4999,1.7556,0.5008,29.0964,0.4991,3.1121


In [20]:
# show full columns
pd.set_option('display.max_columns', None)
this_table = full_table.loc[table_flex.index].drop(columns = ['tabfast_time', f'tabfast_{metric}', 'tabflex_time', f'tabflex_{metric}'])
# ['tabpfn', 'tabflex', 'LinearModel', 'DecisionTree', 'XGBoost', 'MLP', 'TabNet', 'NODE', 'STG', 'rtdl_ResNet']
selected_methods = ['CatBoost', 'MLP', 'TabNet', 'rtdl_ResNet', 'tabpfn', 'Ours-2']
# selected_methods = ['tabpfn', 'tabflex', 'NODE', 'STG', 'rtdl_ResNet']
columns = ['num_classes', 'num_features', 'num_instances'] + [f'{method}_{metric}' for method in selected_methods] + [f'{method}_time' for method in selected_methods]
this_table = this_table[columns]
this_table


,num_classes,num_features,num_instances,CatBoost_AUC,MLP_AUC,TabNet_AUC,rtdl_ResNet_AUC,tabpfn_AUC,Ours-2_AUC,CatBoost_time,MLP_time,TabNet_time,rtdl_ResNet_time,tabpfn_time,Ours-2_time
openml__Bioresponse__9910,1.0,1776.0,3751.0,0.8555,0.8293,0.8145,0.8335,0.4980,0.4905,2.6371,6.1119,57.4285,5.8221,1.2862,6.8684
openml__SpeedDating__146607,1.0,120.0,8378.0,0.8637,0.8460,0.8286,0.8626,0.5469,0.8500,12.0404,13.0573,150.6250,11.0074,1.5849,1.8868
openml__audiology__7,24.0,69.0,226.0,0.8752,0.6498,0.6157,0.9229,0.8182,0.8166,54.5559,5.1219,7.4409,4.9215,0.2328,0.2614
openml__cnae-9__9981,9.0,856.0,1080.0,0.9899,0.9964,0.9841,0.9944,0.4794,0.9558,1.7671,10.5023,26.5780,5.5932,0.5066,3.7996
openml__guillermo__168337,1.0,4296.0,20000.0,0.0000,0.7577,0.7076,0.7762,0.4999,0.5008,0.0000,33.6682,340.3815,43.4568,1.7556,29.0964
openml__jasmine__168911,1.0,144.0,2984.0,0.8712,0.8516,0.8226,0.8538,0.7040,0.8552,1.6779,6.0579,61.8220,4.4049,1.2732,0.9906
openml__nomao__9977,1.0,118.0,34465.0,0.9915,0.9913,0.9923,0.9929,0.7623,0.9884,4.2106,86.4505,517.7118,70.0467,1.8179,5.3433


In [21]:
this_table = this_table.astype({'num_classes': int, 'num_features': int, 'num_instances': int})
# for the index, only keep the str between two "__"s
this_table.index = this_table.index.map(lambda x: x.split('__')[1])
# for the columns, replace the "_" with " "
this_table.columns = this_table.columns.str.replace('_', ' ')
this_table.columns = this_table.columns.str.replace('rtdl ', '')
# for floats, round to 3 decimal places when convert the talbe to latex
# make all columns align to the center
print(this_table.to_latex(float_format="%.3f", column_format='c'*len(this_table.columns), escape=False))

\begin{tabular}{ccccccccccccccc}
\toprule
 & num classes & num features & num instances & CatBoost AUC & MLP AUC & TabNet AUC & ResNet AUC & tabpfn AUC & Ours-2 AUC & CatBoost time & MLP time & TabNet time & ResNet time & tabpfn time & Ours-2 time \\
\midrule
Bioresponse & 1 & 1776 & 3751 & 0.856 & 0.829 & 0.815 & 0.834 & 0.498 & 0.490 & 2.637 & 6.112 & 57.428 & 5.822 & 1.286 & 6.868 \\
SpeedDating & 1 & 120 & 8378 & 0.864 & 0.846 & 0.829 & 0.863 & 0.547 & 0.850 & 12.040 & 13.057 & 150.625 & 11.007 & 1.585 & 1.887 \\
audiology & 24 & 69 & 226 & 0.875 & 0.650 & 0.616 & 0.923 & 0.818 & 0.817 & 54.556 & 5.122 & 7.441 & 4.921 & 0.233 & 0.261 \\
cnae-9 & 9 & 856 & 1080 & 0.990 & 0.996 & 0.984 & 0.994 & 0.479 & 0.956 & 1.767 & 10.502 & 26.578 & 5.593 & 0.507 & 3.800 \\
guillermo & 1 & 4296 & 20000 & 0.000 & 0.758 & 0.708 & 0.776 & 0.500 & 0.501 & 0.000 & 33.668 & 340.382 & 43.457 & 1.756 & 29.096 \\
jasmine & 1 & 144 & 2984 & 0.871 & 0.852 & 0.823 & 0.854 & 0.704 & 0.855 & 1.678 & 6.058 & 61

In [22]:
this_table = full_table.loc[table_fast.index].drop(columns = ['tabflex_time', f'tabflex_{metric}', 'tabfast_time', f'tabfast_{metric}'])
# ['tabpfn', 'tabflex', 'LinearModel', 'DecisionTree', 'XGBoost', 'MLP', 'TabNet', 'NODE', 'STG', 'rtdl_ResNet']
selected_methods = ['CatBoost', 'MLP', 'TabNet', 'rtdl_ResNet', 'tabpfn', 'Ours-2', ]
columns = ['num_classes', 'num_features', 'num_instances'] + [f'{method}_{metric}' for method in selected_methods] + [f'{method}_time' for method in selected_methods]
this_table = this_table[columns]
this_table

,num_classes,num_features,num_instances,CatBoost_AUC,MLP_AUC,TabNet_AUC,rtdl_ResNet_AUC,tabpfn_AUC,Ours-2_AUC,CatBoost_time,MLP_time,TabNet_time,rtdl_ResNet_time,tabpfn_time,Ours-2_time
openml__MiniBooNE__168335,1.0,50.0,130064.0,0.9778,0.9782,0.927,0.9692,0.9769,0.9740,2.6046,265.5586,699.0828,447.4033,3.1937,7.2222
openml__airlines__189354,1.0,7.0,539383.0,0.6977,0.6509,0.000,0.7047,0.6279,0.0000,4.5096,511.9868,0.0000,571.4248,9.7280,0.0000
openml__albert__189356,1.0,78.0,425240.0,0.7677,0.6542,0.000,0.7562,0.6860,0.0000,33.9801,405.5280,0.0000,490.1059,9.3899,0.0000
openml__higgs__146606,1.0,28.0,98050.0,0.7871,0.7931,0.000,0.8119,0.7241,0.7560,1.9992,222.4406,0.0000,199.2417,2.8184,4.9200
openml__poker-hand__9890,10.0,10.0,1025009.0,0.8385,0.5393,0.000,0.0000,0.7176,0.8372,355.3651,504.5200,0.0000,0.0000,15.3640,4.8790


In [23]:
this_table = this_table.astype({'num_classes': int, 'num_features': int, 'num_instances': int})
# for the index, only keep the str between two "__"s
this_table.index = this_table.index.map(lambda x: x.split('__')[1])
# for the columns, replace the "_" with " "
this_table.columns = this_table.columns.str.replace('_', ' ')
this_table.columns = this_table.columns.str.replace('rtdl ', '')
# for floats, round to 3 decimal places when convert the talbe to latex
# make all columns align to the center
print(this_table.to_latex(float_format="%.3f", column_format='c'*len(this_table.columns), escape=False))

\begin{tabular}{ccccccccccccccc}
\toprule
 & num classes & num features & num instances & CatBoost AUC & MLP AUC & TabNet AUC & ResNet AUC & tabpfn AUC & Ours-2 AUC & CatBoost time & MLP time & TabNet time & ResNet time & tabpfn time & Ours-2 time \\
\midrule
MiniBooNE & 1 & 50 & 130064 & 0.978 & 0.978 & 0.927 & 0.969 & 0.977 & 0.974 & 2.605 & 265.559 & 699.083 & 447.403 & 3.194 & 7.222 \\
airlines & 1 & 7 & 539383 & 0.698 & 0.651 & 0.000 & 0.705 & 0.628 & 0.000 & 4.510 & 511.987 & 0.000 & 571.425 & 9.728 & 0.000 \\
albert & 1 & 78 & 425240 & 0.768 & 0.654 & 0.000 & 0.756 & 0.686 & 0.000 & 33.980 & 405.528 & 0.000 & 490.106 & 9.390 & 0.000 \\
higgs & 1 & 28 & 98050 & 0.787 & 0.793 & 0.000 & 0.812 & 0.724 & 0.756 & 1.999 & 222.441 & 0.000 & 199.242 & 2.818 & 4.920 \\
poker-hand & 10 & 10 & 1025009 & 0.839 & 0.539 & 0.000 & 0.000 & 0.718 & 0.837 & 355.365 & 504.520 & 0.000 & 0.000 & 15.364 & 4.879 \\
\bottomrule
\end{tabular}



In [30]:
top_n = 5

# Create a new table with the required information
new_table = pd.DataFrame()

# Get the datasets from table_flex
datasets = table_flex.index.map(lambda x: x.split('__')[1])
new_table['dataset'] = datasets
new_table['num classes'] = full_table.loc[table_flex.index, 'num_classes'].values
new_table['num features'] = full_table.loc[table_flex.index, 'num_features'].values
new_table['num instances'] = full_table.loc[table_flex.index, 'num_instances'].values

# Calculate the mean AUC of the top 5 algorithms
baseline_columns = [col for col in full_table.columns if f'{metric}' in col and 'Ours-2' not in col and 'tabfast' not in col and 'tabflex' not in col and 'tabsmall' not in col and 'tabpfn' not in col]
top_n_auc = full_table.loc[table_flex.index, baseline_columns].apply(lambda row: sorted(row[row!=0], reverse=True)[top_n-1], axis=1).values
new_table[f'top {top_n} mean AUC'] = top_n_auc

# Get TabPFN AUC
new_table['TabPFN AUC'] = full_table.loc[table_flex.index, f'tabpfn_{metric}'].values

# Get Ours-2 AUC
new_table['Ours-2 AUC'] = full_table.loc[table_flex.index, f'tabflex_{metric}'].values

# Calculate the mean time of the top 5 algorithms
time_columns = [col for col in full_table.columns if 'time' in col and 'Ours-2' not in col and 'tabfast' not in col and 'tabflex' not in col]
top_n_time = full_table.loc[table_flex.index, time_columns].apply(lambda row: sorted(row[row!=0])[top_n-1], axis=1).values
new_table[f'top {top_n} mean time'] = top_n_time

# Get TabPFN time
new_table['TabPFN time'] = full_table.loc[table_flex.index, 'tabpfn_time'].values

# Get Ours-2 time
new_table['Ours-2 time'] = full_table.loc[table_flex.index, 'Ours-2_time'].values

# Display the new table
new_table

,dataset,num classes,num features,num instances,top 5 mean AUC,TabPFN AUC,Ours-2 AUC,top 5 mean time,TabPFN time,Ours-2 time
0,Bioresponse,1.0,1776.0,3751.0,0.8464,0.4980,0.4905,2.4921,1.2862,6.8684
1,SpeedDating,1.0,120.0,8378.0,0.8626,0.5469,0.8500,1.5849,1.5849,1.8868
2,cnae-9,9.0,856.0,1080.0,0.9964,0.4794,0.9558,0.5112,0.5066,3.7996
3,guillermo,1.0,4296.0,20000.0,0.7481,0.4999,0.5008,33.6682,1.7556,29.0964
4,jasmine,1.0,144.0,2984.0,0.8604,0.7040,0.8552,0.6801,1.2732,0.9906
5,nomao,1.0,118.0,34465.0,0.9923,0.7623,0.9884,4.0291,1.8179,5.3433


In [31]:
# new table to latex, keep 2 decimal places
print(new_table.to_latex(float_format="%.2f", column_format='c'*len(new_table.columns), escape=False, index=False))

\begin{tabular}{cccccccccc}
\toprule
dataset & num classes & num features & num instances & top 5 mean AUC & TabPFN AUC & Ours-2 AUC & top 5 mean time & TabPFN time & Ours-2 time \\
\midrule
Bioresponse & 1.00 & 1776.00 & 3751.00 & 0.85 & 0.50 & 0.49 & 2.49 & 1.29 & 6.87 \\
SpeedDating & 1.00 & 120.00 & 8378.00 & 0.86 & 0.55 & 0.85 & 1.58 & 1.58 & 1.89 \\
cnae-9 & 9.00 & 856.00 & 1080.00 & 1.00 & 0.48 & 0.96 & 0.51 & 0.51 & 3.80 \\
guillermo & 1.00 & 4296.00 & 20000.00 & 0.75 & 0.50 & 0.50 & 33.67 & 1.76 & 29.10 \\
jasmine & 1.00 & 144.00 & 2984.00 & 0.86 & 0.70 & 0.86 & 0.68 & 1.27 & 0.99 \\
nomao & 1.00 & 118.00 & 34465.00 & 0.99 & 0.76 & 0.99 & 4.03 & 1.82 & 5.34 \\
\bottomrule
\end{tabular}



In [26]:
# Create a new table with the required information
new_table = pd.DataFrame()

# Get the datasets from table_fast
datasets = table_fast.index.map(lambda x: x.split('__')[1])
new_table['dataset'] = datasets
new_table['num classes'] = full_table.loc[table_fast.index, 'num_classes'].values
new_table['num features'] = full_table.loc[table_fast.index, 'num_features'].values
new_table['num instances'] = full_table.loc[table_fast.index, 'num_instances'].values

# Calculate the mean AUC of the top 5 algorithms
baseline_columns = [col for col in full_table.columns if f'{metric}' in col and 'Ours-2' not in col and 'tabfast' not in col and 'tabflex' not in col and 'tabpfn' not in col]
top_5_auc = full_table.loc[table_fast.index, baseline_columns].apply(lambda row: sorted(row[row!=0], reverse=True)[4], axis=1).values
new_table['top 5 mean AUC'] = top_5_auc

# Get TabPFN AUC
new_table['TabPFN AUC'] = full_table.loc[table_fast.index, f'tabpfn_{metric}'].values

# Get Ours-2 AUC
new_table['Ours-2 AUC'] = full_table.loc[table_fast.index, f'tabfast_{metric}'].values

# Calculate the mean time of the top 5 algorithms
time_columns = [col for col in full_table.columns if 'time' in col and 'Ours-2' not in col and 'tabfast' not in col and 'tabflex' not in col]
top_5_time = full_table.loc[table_fast.index, time_columns].apply(lambda row: sorted(row[row!=0])[4], axis=1).values
new_table['top 5 mean time'] = top_5_time

# Get TabPFN time
new_table['TabPFN time'] = full_table.loc[table_fast.index, 'tabpfn_time'].values

# Get Ours-2 time
new_table['Ours-2 time'] = full_table.loc[table_fast.index, 'Ours-2_time'].values

# Display the new table
new_table

,dataset,num classes,num features,num instances,top 5 mean AUC,TabPFN AUC,Ours-2 AUC,top 5 mean time,TabPFN time,Ours-2 time
0,MiniBooNE,1.0,50.0,130064.0,0.9776,0.9769,0.9740,10.7980,3.1937,7.2222
1,airlines,1.0,7.0,539383.0,0.6977,0.6279,0.6393,6.5326,9.7280,0.0000
2,albert,1.0,78.0,425240.0,0.7087,0.6860,0.7011,33.9801,9.3899,0.0000
3,higgs,1.0,28.0,98050.0,0.7871,0.7241,0.7560,3.4619,2.8184,4.9200
4,poker-hand,10.0,10.0,1025009.0,0.5383,0.7176,0.8372,504.5200,15.3640,4.8790


In [27]:
# new table to latex, keep 2 decimal places
print(new_table.to_latex(float_format="%.2f", column_format='c'*len(new_table.columns), escape=False, index=False))

\begin{tabular}{cccccccccc}
\toprule
dataset & num classes & num features & num instances & top 5 mean AUC & TabPFN AUC & Ours-2 AUC & top 5 mean time & TabPFN time & Ours-2 time \\
\midrule
MiniBooNE & 1.00 & 50.00 & 130064.00 & 0.98 & 0.98 & 0.97 & 10.80 & 3.19 & 7.22 \\
airlines & 1.00 & 7.00 & 539383.00 & 0.70 & 0.63 & 0.64 & 6.53 & 9.73 & 0.00 \\
albert & 1.00 & 78.00 & 425240.00 & 0.71 & 0.69 & 0.70 & 33.98 & 9.39 & 0.00 \\
higgs & 1.00 & 28.00 & 98050.00 & 0.79 & 0.72 & 0.76 & 3.46 & 2.82 & 4.92 \\
poker-hand & 10.00 & 10.00 & 1025009.00 & 0.54 & 0.72 & 0.84 & 504.52 & 15.36 & 4.88 \\
\bottomrule
\end{tabular}



In [28]:
# Create a new table with the required information
combined_table = pd.DataFrame()

# Get datasets from both table_flex and table_fast
datasets = pd.Series(list(table_flex.index)+list(table_fast.index))
combined_table['dataset'] = datasets

# Add #classes, #features, #instances
combined_table['#classes'] = full_table.loc[datasets, 'num_classes'].values
combined_table['#features'] = full_table.loc[datasets, 'num_features'].values
combined_table['#instances'] = full_table.loc[datasets, 'num_instances'].values

# Function to get rank
def get_rank(row, metric):
    # Filter out tabflex, ours-1, tabsmall, tabfast
    filtered_row = row.drop([col for col in row.index if any(x in col for x in ['tabflex', 'ours-1', 'tabsmall', 'tabfast'])])
    sorted_values = filtered_row.sort_values(ascending=False)
    return pd.Series({
        f'TabPFN_{metric}_rank': sorted_values.index.get_loc(f'tabpfn_{metric}') + 1,
        f'Ours-2_{metric}_rank': sorted_values.index.get_loc(f'Ours-2_{metric}') + 1
    })

# Get AUC ranks
auc_columns = [col for col in full_table.columns if f'{metric}' in col]
auc_ranks = full_table.loc[datasets, auc_columns].apply(get_rank, metric=metric, axis=1)
combined_table['TabPFN_AUC_rank'] = auc_ranks['TabPFN_AUC_rank'].values
combined_table['Ours-2_AUC_rank'] = auc_ranks['Ours-2_AUC_rank'].values

# Get time ranks
time_columns = [col for col in full_table.columns if 'time' in col]
time_ranks = full_table.loc[datasets, time_columns].apply(get_rank, metric='time', axis=1)
combined_table['TabPFN_time_rank'] = time_ranks['TabPFN_time_rank'].values
combined_table['Ours-2_time_rank'] = time_ranks['Ours-2_time_rank'].values

# Display the combined table
combined_table


,dataset,#classes,#features,#instances,TabPFN_AUC_rank,Ours-2_AUC_rank,TabPFN_time_rank,Ours-2_time_rank
0,openml__Bioresponse__9910,1.0,1776.0,3751.0,19,20,17,11
1,openml__SpeedDating__146607,1.0,120.0,8378.0,23,9,19,18
2,openml__audiology__7,24.0,69.0,226.0,12,13,15,14
3,openml__cnae-9__9981,9.0,856.0,1080.0,19,17,16,12
4,openml__guillermo__168337,1.0,4296.0,20000.0,13,12,14,10
5,openml__jasmine__168911,1.0,144.0,2984.0,23,9,16,18
6,openml__nomao__9977,1.0,118.0,34465.0,19,12,17,13
7,openml__MiniBooNE__168335,1.0,50.0,130064.0,6,8,13,12
8,openml__airlines__189354,1.0,7.0,539383.0,12,23,8,23
9,openml__albert__189356,1.0,78.0,425240.0,8,23,8,15
